
# Работа с данными бизнеса в PySpark

    Выполнил: Калинина Ольга Владимировна


## Содержание:

    1  Содержание:
    2  ВВедение
    3  Описание данных
    4  Шаг 1. Загрузка данных и знакомство с ними
        4.1  Загрузка данных об активности пользователей сервиса Яндекс Книги:
        4.2  Знакомство с набором данных:
        4.3  Сравнение структуры таблиц с описанием данных. Проверка типов столбцов и оценка их корректности:
        4.3.1  Вывод созданную Spark схемы данных:
        4.4  Исправление названий столбцов и типов данных
        4.4.1  Исправление таблицы audition
        4.4.2  Исправление таблицы content
        4.5  Анализ объём данных и количество строк в каждой из таблиц:
    5  Шаг 2. Трансформация и преобразование таблиц
        5.1  Создание нового столбца minutes_sessions_long
        5.2  Добавление нового столбца is_weekend
        5.3  Сумма часов сессии для выходных и будних дней
        5.4  Анализ данных по суммарной длительности сессий для взрослого и невзрослого контента
        5.5  Анализ результатов
        5.6  Полный код Шага 2
    6  Шаг 3. Соединение таблиц
        6.1  Обьединение таблиц audition_df и content_df
        6.2  Удаление лишних столбцов
        6.3  Количество пользователей (puid)
        6.4  Уникальные значения поля main_content_type.
        6.5  Полный код Шага 3
    7  Выводы

## ВВедение

        Необходимо обработать и проанализировать реальные пользовательские данные таблиц  (source_db.audition) и (source_db.content) из сервиса Яндекс Книги с помощью PySpark. Таблицы содержат контент разных форматов, включая текст, аудио и не только за период с 1 сентября по 11 декабря 2024 года.
        Руководство сервиса хочет лучше понимать поведение пользователей: какие типы контента они выбирают, как долго его слушают или читают, а также в какие дни недели и через какие платформы (мобильное приложение, веб-версия) это происходит. Эти инсайты позволят улучшить систему рекомендаций и принимать стратегические решения по развитию продукта.
        Для этого необходимо построить агрегаты и сделать бизнес-выводы на их основе. Также нужно проанализировать, как различается суммарное время потребления контента в выходные и будние дни; выяснить, для какого типа контента наблюдается такая разница — для взрослого или невзрослого. Дополнительно просят преобразовать набор данных и записать его в ClickHouse, чтобы отдел аналитики смог проводить свой анализ на очищенных данных.

## Описание данных

   Таблица source_db.audition содержит информацию о пользователях. Есть информация о платформе приложения, времени сессии, времени использования приложения, категории приложения (18+), о городе и стране читателя. 
   Таблица **bookmate.audition** содержит данные об активности пользователей и включает столбцы:
    
    audition_id — уникальный идентификатор сессии чтения или прослушивания;
    puid — идентификатор пользователя;
    usage_platform_ru — название платформы, с помощью которой пользователь взаимодействует с контентом;
    msk_business_dt_str — дата и время события (строка, часовой пояс — МСК);
    app_version — версия приложения;
    adult_content_flg — значение, которое показывает, был ли контент для взрослых ( True или False );
    hours — длительность сессии чтения или прослушивания в часах;
    hours_sessions_long — длительность длинных сессий в часах;
    kids_content_flg — значение, которое показывает, был ли это детский контент ( True или False );
    main_content_id — идентификатор основного контента;
    usage_geo_id — идентификатор географического местоположения пользователя.
    
    Типы данных в таблице source_db.audition;
    name               |type            |
    -------------------+----------------+
    audition_id        |Int32           |
    puid               |String          |
    usage_platform_ru  |String          |
    msk_business_dt_str|Date            |
    app_version        |Nullable(String)|
    adult_content_flg  |Bool            |
    hours              |Float32         |
    hours_sessions_long|Float32         |
    kids_content_flg   |Bool            |
    main_content_id    |String          |
    usage_geo_id_name  |String          |
    usage_country_name |String          |
    ------------------------------------+
    
    Таблица содержит информацию о контенте. Его название, тип контента, название контента, длительность контента, автор контента, теги с типами жанров.
     Таблица **bookmate.content** включает столбцы:
    
    main_content_id — идентификатор основного контента;
    main_author_id — идентификатор основного автора контента;
    main_content_type — тип контента: аудио, текст или другой;
    main_content_name — название контента;
    main_content_duration_hours — длительность контента в часах;
    published_topic_title_list — список жанров или тем контента.
    
    Типы данных в таблице source_db.content;
    name                       |type         |
    ---------------------------+-------------+
    main_content_id            |String       |
    main_content_type          |String       |
    main_content_name          |String       |
    main_content_duration_hours|Float32      |
    main_author_name           |String       |
    published_topic_title_list |Array(String)|
    -----------------------------------------+

## Шаг 1. Загрузка данных и знакомство с ними

### Загрузка данных об активности пользователей сервиса Яндекс Книги:


In [ ]:
#Загружаем библиатеку:
from pyspark.sql import SparkSession

#Создаём Spark-сессию
spark = (SparkSession.builder
         .appName("DataFrame Basics")
         .config("fs.s3a.endpoint", "storage.yandexcloud.net")
         .getOrCreate())


### Знакомство с набором данных:


In [ ]:
#Указываем пути к файлам, с которыми будем работать в дальнейшем
audition = "s3a://da-plus-dags/data/audition.csv"

#Считываем CSV-файл
audition_csv = (spark.read
                    .option("header", "true")        #Берём названия столбцов из первой строки CSV-файла
                    .option("inferSchema", "true")   #Предлагаем Spark определить схему самостоятельно
                    .csv(audition))


In [ ]:
# Выводим по 10 строк каждой таблицы для ознакомления
print("Таблица audition (первые 10 строк)")
audition_df.show(10)


In [ ]:
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Таблица audition (первые 10 строк)                                                                                                                                         |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| +-----+---+------------------------------------+---------------+----------+----------+-----+-------------------+-------------------+-----+--------+--------------------+---------+ |
| |  162|  0|68296628-f9d6-11ef-be00-c2c9fa6fd3d5|        Станция|2024-11-26|       _c5|False|0.03777777777777777|0.03777777777777778| True|oCURrBKV|              Алматы|Казахстан| |
| +-----+---+------------------------------------+---------------+----------+----------+-----+-------------------+-------------------+-----+--------+--------------------+---------+ |
| |  213|  1|                682966dc-f9d6-11e...|        Станция|2024-11-26|      NULL|false|  8.333333333333E-4|                0.0| true|qOL0JJL5|              Москва|   Россия| |
| |   63|  2|                682966dc-f9d6-11e...|        Станция|2024-11-26|      NULL|false| 0.0044444444444444|                0.0| true|ndM5nzgT|             Иркутск|   Россия| |
| |    2|  4|                68296704-f9d6-11e...|        Станция|2024-11-26|      NULL| true| 0.0016666666666666|                0.0|false|HW5y0HqU|     Санкт-Петербург|   Россия| |
| |   28|  6|                68296722-f9d6-11e...|        Станция|2024-11-26|      NULL|false| 0.0069444444444444|                0.0| true|DDP9Jenb|           Махачкала|   Россия| |
| |   38|  7|                68296740-f9d6-11e...|     Музыка iOS|2024-11-26|697.165724| true| 0.4890794444444444| 0.4890794444444444|false|R7tyI6r6|           Волгоград|   Россия| |
| |11162|  8|                68296740-f9d6-11e...|     Музыка iOS|2024-11-26|698.166333| true| 0.1847552777777777|          0.1847275|false|R7tyI6r6|Свердловская область|   Россия| |
| |11119|  9|                6829675e-f9d6-11e...|Букмейт Android|2024-11-26|       6.7| true| 1.4530555555555555| 1.4530555555555555|false|QBFiEtWi|Республика Татарстан|   Россия| |
| |   47| 10|                6829677c-f9d6-11e...|Букмейт Android|2024-11-26|       6.5| true| 0.5399818181818182| 0.5399818181818182|false|xnwMA9a7|     Нижний Новгород|   Россия| |
| |  194| 13|                6829679a-f9d6-11e...|        Станция|2024-11-26|      NULL| true| 0.0197222222222222|                0.0| true|RrDJWAfV|             Саратов|   Россия| |
| |  213| 15|                682967b8-f9d6-11e...|Букмейт Android|2024-11-26|       6.7| true| 0.2905090909090909| 0.2905090909090909|false|jxNgtTqG|              Москва|   Россия| |
| +-----+---+------------------------------------+---------------+----------+----------+-----+-------------------+-------------------+-----+--------+--------------------+---------+ |
| only showing top 10 rows 

Как видно из расчётов, в таблице audition содержимое столбцов и их тип не совподает с предпологаемым. В таблице 12 столбцов и 1002895 строк. Но названия в столбцах отсутствует и лишний один столбец (первый).

In [ ]:

#Указываем пути к файлам, с которыми будем работать в дальнейшем
content = "s3a://da-plus-dags/data/content.csv"

#Считываем CSV-файл
content_csv = (spark.read
                    .option("header", "true")        #Берём названия столбцов из первой строки CSV-файла
                    .option("inferSchema", "true")   #Предлагаем Spark определить схему самостоятельно
                    .csv(content))

In [ ]:

print("Таблица content (первые 10 строк)")
content_df.show(10)

In [ ]:
| Таблица content (первые 10 строк)                                                                          
| +--------+---------+------------------------------------------+----------+--------------------+--------------------+
| |A00hxVIL|     Book|Давай поговорим о твоих доходах и расходах|     4.151|'Синхронизировано', |       Карл Ричардс |
|                                                               |          |     'Саморазвитие',|                    |
|                                                               |          |   'Личные финансы',|                    |
|                                                               |          |     'Бизнес'       |                    |
| +--------+---------+------------------------------------------+----------+--------------------+--------------------+
| |A03if3j5|Comicbook|                      Минимализм из ком...| 2.6854546|'Уборка и организ...|Элизабет Энрайт Ф...|
| |A05XPJgr|     Book|                       Гоните ваши денежки| 6.9114366|'Художественная л...|Наталья Александрова|
| |A06fBP22|     Book|                      Крайон. Судьбу мо...| 3.5218182|'Эзотерика', 'Окк...|        Тамара Шмидт|
| |A0FkgwIl|Comicbook|                      Майор Гром. Допол...|0.20363636|           'Комиксы'|    Артём Габрелянов|
| |A0OKAjnf|Audiobook|                      Радикальное Проще...| 5.0780554|'Психология', 'Са...|       Колин Типпинг|
| |A0e7vsJT|     Book|                      Обоняние. Увлекат...|  8.896182|'Наука', 'Психоло...|        Паоло Пелоси|
| |A0fAe01q|Audiobook|                             Амулет ведьмы| 10.473333|'Художественная л...|     Анна Безбрежная|
| |A0mSkD2g|Audiobook|                      Правила инвестиро...|  9.809444|'Биографии и мему...|      Джереми Миллер|
| |A0oeFBMU|Audiobook|                      Закон трех отрицаний| 14.834167|'Художественная л...| Александра Маринина|
| |A0s12uzA|Audiobook|                      Женщина, которая ...|  9.502778|'Художественная л...|    Андрей Бронников|
| +--------+---------+------------------------------------------+----------+--------------------+--------------------+
| only showing top 10 rows 

Как видно из расчётов, в таблице content содержимое столбцов и их тип не совподает с предпологаемым. В таблице 6 столбцов и 31667  строк. Но названия у столбцов отсутствуют. 

### Сравнение структуры таблиц с описанием данных. Проверка типов столбцов и оценка их корректности: 


#### Вывод созданную Spark схемы данных:


In [ ]:
# Выводим схему таблицы для анализа данных
print("Схема таблицы audition")
audition_df.printSchema()

In [ ]:
| Схема таблицы audition                                                                                                                                                    |
| root                                                                                                                                                                               |
| |-- 162: integer (nullable = true)                                                                                                                                                 |
| |-- 0: integer (nullable = true)                                                                                                                                                   |
| |-- 68296628-f9d6-11ef-be00-c2c9fa6fd3d5: string (nullable = true)                                                                                                                 |
| |-- Станция: string (nullable = true)                                                                                                                                              |
| |-- 2024-11-26: date (nullable = true)                                                                                                                                             |
| |-- _c5: string (nullable = true)                                                                                                                                                  |
| |-- False: boolean (nullable = true)                                                                                                                                               |
| |-- 0.03777777777777777: double (nullable = true)                                                                                                                                  |
| |-- 0.03777777777777778: double (nullable = true)                                                                                                                                  |
| |-- True: boolean (nullable = true)                                                                                                                                                |
| |-- oCURrBKV: string (nullable = true)                                                                                                                                             |
| |-- Алматы: string (nullable = true)                                                                                                                                               |
| |-- Казахстан: string (nullable = true)                                                                                                                                            |
|                             

In [ ]:
Типы данных в таблице source_db.audition;
name               |type            |
-------------------+----------------+
audition_id        |Int32           |
puid               |String          |
usage_platform_ru  |String          |
msk_business_dt_str|Date            |
app_version        |Nullable(String)|
adult_content_flg  |Bool            |
hours              |Float32         |
hours_sessions_long|Float32         |
kids_content_flg   |Bool            |
main_content_id    |String          |
usage_geo_id_name  |String          |
usage_country_name |String          |
------------------------------------+

 В таблице audition есть лишний столбец с типом integer (nullable = true). И нет названий у сталбцов. Также не совпадают типы данных в столбцах  **hours** (ожидался: Float32, а фактически: double (Float64)), **hours_sessions_long**(ожидался: Float32, а фактически: double (Float64)), **app_version**(ожидался: Nullable(String), фактически: string).

In [ ]:
# Выводим схему таблицы для анализа данных
print("Схема таблицы content")
content_df.printSchema()

In [ ]:
| Схема таблицы content                                                                                                                                                      |
| root                                                                                                                                                                               |
| |-- A00hxVIL: string (nullable = true)                                                                                                                                             |
| |-- Book: string (nullable = true)                                                                                                                                                 |
| |-- Давай поговорим о твоих доходах и расходах: string (nullable = true)                                                                                                           |
| |-- 4.151: double (nullable = true)                                                                                                                                                |
| |-- 'Синхронизировано', 'Бизнес', 'Саморазвитие', 'Личные финансы': string (nullable = true)                                                                                       |
| |-- Карл Ричардс: string (nullable = true)  

In [ ]:
Типы данных в таблице source_db.content;
name                       |type         |
---------------------------+-------------+
main_content_id            |String       |
main_content_type          |String       |
main_content_name          |String       |
main_content_duration_hours|Float32      |
main_author_name           |String       |
published_topic_title_list |Array(String)|
-----------------------------------------+

В таблице content типы данных у столбцов  **main_content_duration_hours** (ожидался Float32, а фактически double), для **published_topic_title_list**(ожидался Array(String), а фактически string)) не совпадают. И также нет названий у столбцов. И столбцы **main_author_name** и **published_topic_title_list** поменяны местами.

### Исправление названий столбцов и типов данных

#### Исправление таблицы audition

In [ ]:
# Загружаем библиотеку, для создания сессии PySpark:
from pyspark.sql import SparkSession
# Загружаем библиотеку для замены типа данных в столбцах:
from pyspark.sql.types import FloatType
# Загружаем библиотеку для создания функций в PySpark:
from pyspark.sql import functions as F


# Создаём Spark-сессию
spark = (SparkSession.builder
         .appName("DataFrame Basics")
         .config("fs.s3a.endpoint", "storage.yandexcloud.net")
         .getOrCreate())

# Указываем пути к файлам
audition = "s3a://da-plus-dags/data/audition.csv"
content = "s3a://da-plus-dags/data/content.csv"

# Считываем CSV-файл
audition_df = (spark.read
    .option("header", "false")
    .option("inferSchema", "true")
    .csv(audition))

# Считываем CSV-файл content
content_df = (spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(content))

# Новые названия для столбцов (без первого)
new_column_names = [
    "audition_id",
    "puid",
    "usage_platform_ru", 
    "msk_business_dt_str",
    "app_version",
    "adult_content_flg",
    "hours",
    "hours_sessions_long",
    "kids_content_flg",
    "main_content_id", 
    "usage_geo_id_name",
    "usage_country_name"
]

# Удаляем первый столбец (_c0) и переименовываем остальные
audition_df_new = audition_df.select(
    audition_df.columns[1:]  # все столбцы кроме первого (_c0)
).toDF(*new_column_names)

audition_df = audition_df_new \
    .withColumn("hours", F.col("hours").cast(FloatType())) \
    .withColumn("hours_sessions_long", F.col("hours_sessions_long").cast(FloatType()))

# Показываем результат
print("Обработанная таблица audition_df:")
audition_df.printSchema()
audition_df.show(10)

In [ ]:
# Исправляем данные и выводим результат:
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Обработанная таблица audition_df:                                                                                                                                                                                |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| root                                                                                                                                                                                                             |
| |-- audition_id: integer (nullable = true)                                                                                                                                                                       |
| |-- puid: string (nullable = true)                                                                                                                                                                               |
| |-- usage_platform_ru: string (nullable = true)                                                                                                                                                                  |
| |-- msk_business_dt_str: date (nullable = true)                                                                                                                                                                  |
| |-- app_version: string (nullable = true)                                                                                                                                                                        |
| |-- adult_content_flg: boolean (nullable = true)                                                                                                                                                                 |
| |-- hours: float (nullable = true)                                                                                                                                                                               |
| |-- hours_sessions_long: float (nullable = true)                                                                                                                                                                 |
| |-- kids_content_flg: boolean (nullable = true)                                                                                                                                                                  |
| |-- main_content_id: string (nullable = true)                                                                                                                                                                    |
| |-- usage_geo_id_name: string (nullable = true)                                                                                                                                                                  |
| |-- usage_country_name: string (nullable = true)                                                                                                                                                                 |
|                                                                                                                                                                                                                  |
| +-----------+--------------------+-----------------+-------------------+-----------+-----------------+------------+-------------------+----------------+---------------+--------------------+------------------+ |
| |audition_id|                puid|usage_platform_ru|msk_business_dt_str|app_version|adult_content_flg|       hours|hours_sessions_long|kids_content_flg|main_content_id|   usage_geo_id_name|usage_country_name| |
| +-----------+--------------------+-----------------+-------------------+-----------+-----------------+------------+-------------------+----------------+---------------+--------------------+------------------+ |
| |          0|68296628-f9d6-11e...|          Станция|         2024-11-26|       NULL|            false| 0.037777778|        0.037777778|            true|       oCURrBKV|              Алматы|         Казахстан| |
| |          1|682966dc-f9d6-11e...|          Станция|         2024-11-26|       NULL|            false|8.3333335E-4|                0.0|            true|       qOL0JJL5|              Москва|            Россия| |
| |          2|682966dc-f9d6-11e...|          Станция|         2024-11-26|       NULL|            false|0.0044444446|                0.0|            true|       ndM5nzgT|             Иркутск|            Россия| |
| |          4|68296704-f9d6-11e...|          Станция|         2024-11-26|       NULL|             true|0.0016666667|                0.0|           false|       HW5y0HqU|     Санкт-Петербург|            Россия| |
| |          6|68296722-f9d6-11e...|          Станция|         2024-11-26|       NULL|            false|0.0069444445|                0.0|            true|       DDP9Jenb|           Махачкала|            Россия| |
| |          7|68296740-f9d6-11e...|       Музыка iOS|         2024-11-26| 697.165724|             true|  0.48907945|         0.48907945|           false|       R7tyI6r6|           Волгоград|            Россия| |
| |          8|68296740-f9d6-11e...|       Музыка iOS|         2024-11-26| 698.166333|             true|  0.18475528|          0.1847275|           false|       R7tyI6r6|Свердловская область|            Россия| |
| |          9|6829675e-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.7|             true|   1.4530555|          1.4530555|           false|       QBFiEtWi|Республика Татарстан|            Россия| |
| |         10|6829677c-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.5|             true|  0.53998184|         0.53998184|           false|       xnwMA9a7|     Нижний Новгород|            Россия| |
| |         13|6829679a-f9d6-11e...|          Станция|         2024-11-26|       NULL|             true| 0.019722221|                0.0|            true|       RrDJWAfV|             Саратов|            Россия| |
| +-----------+--------------------+-----------------+-------------------+-----------+-----------------+------------+-------------------+----------------+---------------+--------------------+------------------+ |
| only showing top 10 rows 

После обработки данных столбцы имеют соответствующий тип и названия.

#### Исправление таблицы content

In [ ]:
# Новые названия для столбцов
new_column_name = [
    "main_content_id",
    "main_content_type",
    "main_content_name ", 
    "main_content_duration_hours",
    "published_topic_title_list",
    "main_author_name"
]

# Переименовываем остальные столбцы
content_df = content_df.select(
    content_df.columns[0:]  # все столбцы
).toDF(*new_column_name)

# Преобразуем строку в массив split()
content_df = (content_df
    .withColumn("published_topic_title_list", F.split(F.col("published_topic_title_list"), ",\\s*"))
    .withColumn("main_content_duration_hours", F.col("main_content_duration_hours").cast(FloatType()))
)

# Показываем результат
print("Обработанная таблица content_df:")
content_df.printSchema()
content_df.show(10, truncate=False)

In [ ]:
| Обработанная таблица content_df:                                                                                                                                                                                                                                            |
| root                                                                                                                                                                                                                                                                        |
| |-- main_content_id: string (nullable = true)                                                                                                                                                                                                                               |
| |-- main_content_type: string (nullable = true)                                                                                                                                                                                                                             |
| |-- main_content_name : string (nullable = true)                                                                                                                                                                                                                            |
| |-- main_content_duration_hours: float (nullable = true)                                                                                                                                                                                                                    |
| |-- published_topic_title_list: array (nullable = true)                                                                                                                                                                                                                     |
|                                                                                                                                                                                                                                                                             |
| |-- main_author_name: string (nullable = true)                                                                                                                                                                                                                              |
|                                                                                                                                                                                                                                                                             |
| +---------------+-----------------+----------------------------------------------------------------------------------------------------------+---------------------------+------------------------------------------------------------------------+-----------------------+ |
| |main_content_id|main_content_type|main_content_name                                                                                         |main_content_duration_hours|published_topic_title_list                                              |main_author_name       | |
| +---------------+-----------------+----------------------------------------------------------------------------------------------------------+---------------------------+------------------------------------------------------------------------+-----------------------+ |
| |A00hxVIL       |Book             |Давай поговорим о твоих доходах и расходах                                                                |4.151                      |['Синхронизировано', 'Бизнес', 'Саморазвитие', 'Личные финансы']        |Карл Ричардс           | |
| |A03if3j5       |Comicbook        |Минимализм из комнаты в комнату. Пошаговая система очищения дома от прихожей до спальни                   |2.6854546                  |['Уборка и организация пространства', 'Домашние дела']                  |Элизабет Энрайт Филлипс| |
| |A05XPJgr       |Book             |Гоните ваши денежки                                                                                       |6.9114366                  |['Художественная литература', 'Детективы']                              |Наталья Александрова   | |
| |A06fBP22       |Book             |Крайон. Судьбу можно изменить! Как воплотить в реальность любой сценарий жизни                            |3.5218182                  |['Эзотерика', 'Оккультизм']                                             |Тамара Шмидт           | |
| |A0FkgwIl       |Comicbook        |Майор Гром. Дополнительные материалы к тому №6                                                            |0.20363636                 |['Комиксы']                                                             |Артём Габрелянов       | |
| |A0OKAjnf       |Audiobook        |Радикальное Прощение: родители и дети. Почему так важно простить своих близких и как сделать это правильно|5.0780554                  |['Психология', 'Саморазвитие', 'Аудио']                                 |Колин Типпинг          | |
| |A0e7vsJT       |Book             |Обоняние. Увлекательное погружение в науку о запахах                                                      |8.896182                   |['Наука', 'Психология']                                                 |Паоло Пелоси           | |
| |A0fAe01q       |Audiobook        |Амулет ведьмы                                                                                             |10.473333                  |['Художественная литература', 'Фэнтези', 'Темное фэнтези', 'Аудио']     |Анна Безбрежная        | |
| |A0mSkD2g       |Audiobook        |Правила инвестирования Уоррена Баффетта                                                                   |9.809444                   |['Биографии и мемуары', 'Бизнес', 'Инвестиции', 'Аудио']                |Джереми Миллер         | |
| |A0oeFBMU       |Audiobook        |Закон трех отрицаний                                                                                      |14.834167                  |['Художественная литература', 'Детективы', 'Аудио', 'Полицейская драма']|Александра Маринина    | |
| +---------------+-----------------+----------------------------------------------------------------------------------------------------------+---------------------------+------------------------------------------------------------------------+-----------------------+ |
| only showing top 10 rows                                                                                                                                                                                                                                                    |
|                                                                                                                                                                                                                                                                             |
| +-------+---------------+-----------------+--------------------+---------------------------+--------------------+                                                                                                                                                           |
| |summary|main_content_id|main_content_type|  main_content_name |main_content_duration_hours|    main_author_name|                                                                                                                                                           |
| +-------+---------------+-----------------+--------------------+---------------------------+--------------------+                                                                                                                                                           |
| |  count|          31668|            31668|               31668|                      31662|               31668|                                                                                                                                                           |
| |   mean|           NULL|             NULL|  1939.4655172413793|          8.315491293981303|  2.9800925925925923|                                                                                                                                                           |
| | stddev|           NULL|             NULL|   421.4754680844952|          7.966225030945566|  1.5572836218257042|                                                                                                                                                           |
| |    min|       A00hxVIL|        Audiobook|"Generation ""Р""...|                        0.0|'Классика', 'Худо...|                                                                                                                                                           |
| |    max|       zzzCBlf5|        Comicbook|… нет воспоминани...|                  293.75012|Өтепберген Әлімгерев|                                                                                                                                                           |
| +-------+---------------+-----------------+--------------------+---------------------------+--------------------+                                                                                                                                                           |
|

После обработки данных столбцы имеют соответствующий тип и названия.

### Анализ объём данных и количество строк в каждой из таблиц:

In [ ]:
# Проверяем количество строк в таблице audition_df
audition_count = audition_df.count()

In [ ]:
+-----------+
| 1002895   |
|-----------|

In [ ]:
# Проверяем количество строк в таблице content_df
content_count = content_df.count()

In [ ]:
|-----------|
| 31667     |
+-----------+

    Разница в количестве строк, скорее всего, обусловлено тем, что в таблице audition хранятся данные о пользователях, а в таблице content о контенте. Так как у одной книги может быть сотни и даже сотни тысяч читателей, то и связана одна книга из одной таблицы может с тысячами пользователями из другой. 

## Шаг 2. Трансформация и преобразование таблиц

### Создание нового столбца minutes_sessions_long

Из таблицы audition_df возьмём столбцы puid и hours_sessions_long. Создадим новый столбец minutes_sessions_long, в котором значение hours_sessions_long будет умножено на 60 для расчётов в минутах. А также заменим тип данных этого столбца на int. Выведим первые десять строк полученной таблицы.

In [ ]:
# № 1: Подготовка данных с adult_content_flg и minutes_sessions_long
print("Подготовка данных")

# Включаем в выборку нужные столбцы
step1_df = audition_df.select(
    "puid", 
    "hours_sessions_long", 
    "msk_business_dt_str", 
    "adult_content_flg"
)

# Создаем minutes_sessions_long и обрабатываем пропуски
step1_df = step1_df \
    .withColumn("minutes_sessions_long", (F.col("hours_sessions_long") * 60).cast("int"))

# Выбираем столбцы
step1 = step1_df.select("puid", "hours_sessions_long", "minutes_sessions_long")

print("Первые 10 строк с добавленным столбцом minutes_sessions_long:")
step1.show(10)

In [ ]:
+--------------------+-------------------+---------------------+                |
| |                puid|hours_sessions_long|minutes_sessions_long|              |
| +--------------------+-------------------+---------------------+              |
| |68296628-f9d6-11e...|        0.037777778|                    2|              |
| |682966dc-f9d6-11e...|                0.0|                    0|              |
| |682966dc-f9d6-11e...|                0.0|                    0|              |
| |68296704-f9d6-11e...|                0.0|                    0|              |
| |68296722-f9d6-11e...|                0.0|                    0|              |
| |68296740-f9d6-11e...|         0.48907945|                   29|              |
| |68296740-f9d6-11e...|          0.1847275|                   11|              |
| |6829675e-f9d6-11e...|          1.4530555|                   87|              |
| |6829677c-f9d6-11e...|         0.53998184|                   32|              |
| |6829679a-f9d6-11e...|                0.0|                    0|              |
| +--------------------+-------------------+---------------------+              |
| only showing top 10 rows                                                      |

### Добавление нового столбца is_weekend

К предыдущему запросу добавим новый столбец is_weekend, который покажет, был ли этот день рабочим ( False — рабочий, True — выходной). Выведим первые десять строк полученной таблицы.

In [ ]:
# № 2: Добавляем столбец is_weekend
print(" Добавляем столбец is_weekend ")

# Преобразуем строку с датой в тип date и определяем день недели
step2_df = step1_df.withColumn("date", F.to_date("msk_business_dt_str", "yyyy-MM-dd"))

# Добавляем столбец дня недели (1=воскресенье, 7=суббота в Spark)
step2_df = step2_df.withColumn("day_of_week", F.dayofweek("date"))

# Создаем столбец is_weekend: True для субботы (7) и воскресенья (1), False для остальных дней
step2_df = step2_df.withColumn("is_weekend", 
                              (F.col("day_of_week") == 1) | (F.col("day_of_week") == 7))

step2_df = (
    step2_df.withColumn(
        "is_weekend",
        F.when(F.col("is_weekend") == True, F.lit("Выходной день"))
         .otherwise(F.lit("Рабочий день"))
    )
)

# Выбираем только нужные столбцы для вывода
result_df = step2_df.select("puid", "hours_sessions_long", "minutes_sessions_long", "is_weekend")

print("Первые 10 строк с добавленным столбцом is_weekend:")
result_df.show(10)

In [ ]:
Добавляем столбец is_weekend                                                  |
| Первые 10 строк с добавленным столбцом is_weekend:                            |
| +--------------------+-------------------+---------------------+------------+ |
| |                puid|hours_sessions_long|minutes_sessions_long|  is_weekend| |
| +--------------------+-------------------+---------------------+------------+ |
| |68296628-f9d6-11e...|        0.037777778|                    2|Рабочий день| |
| |682966dc-f9d6-11e...|                0.0|                    0|Рабочий день| |
| |682966dc-f9d6-11e...|                0.0|                    0|Рабочий день| |
| |68296704-f9d6-11e...|                0.0|                    0|Рабочий день| |
| |68296722-f9d6-11e...|                0.0|                    0|Рабочий день| |
| |68296740-f9d6-11e...|         0.48907945|                   29|Рабочий день| |
| |68296740-f9d6-11e...|          0.1847275|                   11|Рабочий день| |
| |6829675e-f9d6-11e...|          1.4530555|                   87|Рабочий день| |
| |6829677c-f9d6-11e...|         0.53998184|                   32|Рабочий день| |
| |6829679a-f9d6-11e...|                0.0|                    0|Рабочий день| |
| +--------------------+-------------------+---------------------+------------+ |
| only showing top 10 rows                                                      |

### Сумма часов сессии для выходных и будних дней

Рассчитаем сумму значений minutes_sessions_long для выходных и будних дней. Назовём этот столбец total_minutes. Отсортируем по is_weekend и сравним значения.

In [ ]:
# № 3: Рассчитываем суммы minutes_sessions_long для выходных и будних дней
print("# № 3: Суммы minutes_sessions_long по типам дней")

# Группируем по is_weekend и считаем сумму
summary_df = step2_df.groupBy("is_weekend") \
    .agg(F.sum("minutes_sessions_long").alias("total_minutes")) \
    .orderBy("is_weekend")

summary_df = summary_df.select("is_weekend", "total_minutes")

print("Суммы minutes_sessions_long по типам дней:")
summary_df.show()

In [ ]:
# № 3: Суммы minutes_sessions_long по типам дней                                |
| Суммы minutes_sessions_long по типам дней:                                    |
| +-------------+-------------+                                                 |
| |   is_weekend|total_minutes|                                                 |
| +-------------+-------------+                                                 |
| |Выходной день|      6599755|                                                 |
| | Рабочий день|     17997378|                                                 |
| +-------------+-------------+                                                 |

Как видно из расчётов, больше всего сумма часов сессий в рабочие дни(17997378), чем в выходные (6599755). Это говорит о том, что пользователи предпочитаеют пользоваться приложением на рабочей неделе, чем на выходных.

### Анализ данных по суммарной длительности сессий для взрослого и невзрослого контента

Проведём анализ отдельно для взрослого и невзрослого контента на основе столбца adult_content_flg. Избавимся от пропусков в итоговой таблице. Отсортируйем данные в таблице сначала по возрастному рейтингу, а затем по выходным. Проанализируйте полученные результаты.

In [ ]:
# № 4: Анализ для взрослого и невзрослого контента
print(" Анализ по adult_content_flg и is_weekend ")

step2_df = (
    step2_df.withColumn(
        "adult_content_flg",
        F.when(F.col("adult_content_flg") == True, F.lit("Взрослый"))
         .otherwise(F.lit("Детский"))
    )
)

# Группируем по adult_content_flg и is_weekend, считаем сумму
result_df = step2_df.groupBy("adult_content_flg", "is_weekend") \
    .agg(F.sum("minutes_sessions_long").alias("total_minutes"))

# Избавляемся от пропусков:
result_df = result_df.dropna()

# Сортируем сначала по adult_content_flg, затем по is_weekend
result_df = result_df.orderBy("adult_content_flg", "is_weekend")

print("Итоговая таблица:")
result_df.dropna().show()

In [ ]:
Анализ по adult_content_flg и is_weekend                                      |
| Итоговая таблица:                                                             |
| +-----------------+-------------+-------------+                               |
| |adult_content_flg|   is_weekend|total_minutes|                               |
| +-----------------+-------------+-------------+                               |
| |         Взрослый|Выходной день|      5198115|                               |
| |         Взрослый| Рабочий день|     14625640|                               |
| |          Детский|Выходной день|      1401640|                               |
| |          Детский| Рабочий день|      3371738|                               |
| +-----------------+-------------+-------------+                               |

### Анализ результатов
    Как видно из расчётов, больше всего сумма часов сессий в рабочие дни(17997378), чем в выходные (6599755). Это говорит о том, что пользователи предпочитаеют пользоваться приложением на рабочей неделе, чем на выходных.
    Также, в результате анализа было выявлено, что суммарное количество часов сессий больше у контента для "взрослых"(19 823 755 часов), чем у контента с рейтингом "детский"(4 773 378 часов). Так результаты расчётов показали, что чаще всего пользователи посещают приложение в будние дни, что с контентом для взрослых(на буднях 14625640 часов, а на выходных 5198115 часов), что для детей (на буднях 3371738 часов, а на выходных 1401640 часов).

### Полный код Шага 2

In [ ]:
# Загружаем библиотеку, для создания сессии PySpark:
from pyspark.sql import SparkSession
# Загружаем библиотеку для замены типа данных в столбцах:
from pyspark.sql.types import (StructType, StructField, LongType, StringType, FloatType)
# Загружаем библиотеку для создания функций в PySpark:
from pyspark.sql import functions as F

# Создаём Spark-сессию
spark = (SparkSession.builder
         .appName("DataFrame Basics")
         .config("fs.s3a.endpoint", "storage.yandexcloud.net")
         .getOrCreate())

# Указываем пути к файлам
audition = "s3a://da-plus-dags/data/audition.csv"


# Считываем CSV-файл
audition_df = (spark.read
    .option("header", "false")
    .option("inferSchema", "true")
    .csv(audition))

# Новые названия для столбцов
new_column_names = [
    "audition_id",
    "puid",
    "usage_platform_ru", 
    "msk_business_dt_str",
    "app_version",
    "adult_content_flg",
    "hours",
    "hours_sessions_long",
    "kids_content_flg",
    "main_content_id", 
    "usage_geo_id_name",
    "usage_country_name"
]

# Удаляем первый столбец (_c0) и переименовываем остальные
audition_df_new = audition_df.select(
    audition_df.columns[1:]  # все столбцы кроме первого (_c0)
).toDF(*new_column_names)

audition_df = audition_df_new \
    .withColumn("hours", F.col("hours").cast(FloatType())) \
    .withColumn("hours_sessions_long", F.col("hours_sessions_long").cast(FloatType()))

# № 1: Подготовка данных с adult_content_flg и minutes_sessions_long
print("Подготовка данных")

# Включаем в выборку нужные столбцы
step1_df = audition_df.select(
    "puid", 
    "hours_sessions_long", 
    "msk_business_dt_str", 
    "adult_content_flg"
)

# Создаем minutes_sessions_long и обрабатываем пропуски
step1_df = step1_df \
    .withColumn("minutes_sessions_long", (F.col("hours_sessions_long") * 60).cast("int"))

# Выбираем столбцы
step1 = step1_df.select("puid", "hours_sessions_long", "minutes_sessions_long")

print("Первые 10 строк с добавленным столбцом minutes_sessions_long:")
step1.show(10)

# № 2: Добавляем столбец is_weekend
print(" Добавляем столбец is_weekend ")

# Преобразуем строку с датой в тип date и определяем день недели
step2_df = step1_df.withColumn("date", F.to_date("msk_business_dt_str", "yyyy-MM-dd"))

# Добавляем столбец дня недели (1=воскресенье, 7=суббота в Spark)
step2_df = step2_df.withColumn("day_of_week", F.dayofweek("date"))

# Создаем столбец is_weekend: True для субботы (7) и воскресенья (1), False для остальных дней
step2_df = step2_df.withColumn("is_weekend", 
                              (F.col("day_of_week") == 1) | (F.col("day_of_week") == 7))

step2_df = (
    step2_df.withColumn(
        "is_weekend",
        F.when(F.col("is_weekend") == True, F.lit("Выходной день"))
         .otherwise(F.lit("Рабочий день"))
    )
)

# Выбираем только нужные столбцы для вывода
result_df = step2_df.select("puid", "hours_sessions_long", "minutes_sessions_long", "is_weekend")

print("Первые 10 строк с добавленным столбцом is_weekend:")
result_df.show(10)

# № 3: Рассчитываем суммы minutes_sessions_long для выходных и будних дней
print("# № 3: Суммы minutes_sessions_long по типам дней")

# Группируем по is_weekend и считаем сумму
summary_df = step2_df.groupBy("is_weekend") \
    .agg(F.sum("minutes_sessions_long").alias("total_minutes")) \
    .orderBy("is_weekend")

summary_df = summary_df.select("is_weekend", "total_minutes")

print("Суммы minutes_sessions_long по типам дней:")
summary_df.show()

# № 4: Анализ для взрослого и невзрослого контента
print(" Анализ по adult_content_flg и is_weekend ")

step2_df = (
    step2_df.withColumn(
        "adult_content_flg",
        F.when(F.col("adult_content_flg") == True, F.lit("Взрослый"))
         .otherwise(F.lit("Детский"))
    )
)

# Группируем по adult_content_flg и is_weekend, считаем сумму
result_df = step2_df.groupBy("adult_content_flg", "is_weekend") \
    .agg(F.sum("minutes_sessions_long").alias("total_minutes"))

# Избавляемся от пропусков:
result_df = result_df.dropna()

# Сортируем сначала по adult_content_flg, затем по is_weekend
result_df = result_df.orderBy("adult_content_flg", "is_weekend")

print("Итоговая таблица:")
result_df.dropna().show()

## Шаг 3. Соединение таблиц

### Обьединение таблиц audition_df и content_df

Объединим таблицы audition_df и content_df по столбцу main_content_id. Проанализируем сколько строк получилось после объединения.

In [ ]:
# Загружаем библиотеку, для создания сессии PySpark:
from pyspark.sql import SparkSession
# Загружаем библиотеку для замены типа данных в столбцах:
from pyspark.sql.types import (StructType, StructField, LongType, StringType, FloatType, ArrayType)
# Загружаем библиотеку для создания функций в PySpark:
from pyspark.sql import functions as F

# Создаём Spark-сессию
spark = (SparkSession.builder
         .appName("DataFrame Basics")
         .config("fs.s3a.endpoint", "storage.yandexcloud.net")
         .getOrCreate())

# Указываем пути к файлам
audition = "s3a://da-plus-dags/data/audition.csv"
content = "s3a://da-plus-dags/data/content.csv"

# Считываем CSV-файл
audition_df = (spark.read
    .option("header", "false")
    .option("inferSchema", "true")
    .csv(audition))

# Считываем CSV-файл content
content_df = (spark.read
    .option("header", "false")
    .option("inferSchema", "true")
    .csv(content))

# Новые названия для столбцов
new_column_names = [
    "audition_id",
    "puid",
    "usage_platform_ru", 
    "msk_business_dt_str",
    "app_version",
    "adult_content_flg",
    "hours",
    "hours_sessions_long",
    "kids_content_flg",
    "main_content_id", 
    "usage_geo_id_name",
    "usage_country_name"
]

# Удаляем первый столбец (_c0) и переименовываем остальные
audition_df_new = audition_df.select(
    audition_df.columns[1:]  # все столбцы кроме первого (_c0)
).toDF(*new_column_names)

audition_df = audition_df_new \
    .withColumn("hours", F.col("hours").cast(FloatType())) \
    .withColumn("hours_sessions_long", F.col("hours_sessions_long").cast(FloatType()))

# Показываем результат
print("Обработанная таблица audition_df:")
audition_df.printSchema()
audition_df.show(10, truncate=False)

# Базовая статистика
audition_df.describe().show(10)

# Новые названия для столбцов
new_column_name = [
    "main_content_id",
    "main_content_type",
    "main_content_name ", 
    "main_content_duration_hours",
    "published_topic_title_list",
    "main_author_name"
]

# Переименовываем остальные столбцы
content_df = content_df.select(
    content_df.columns[0:]  # все столбцы
).toDF(*new_column_name)

# Показываем результат
print("Обработанная таблица content_df:")
content_df.printSchema()
content_df.show(10, truncate=False)

# Базовая статистика
content_df.describe().show(10)

# Присоединяем таблицы типом inner Join,
#который возвращает строки из левого датафрейма, 
#которые имеют совпадения в правом, при этом не включая данные из правого. 

df = audition_df.join(content_df, 
                                        on="main_content_id", 
                                        how="inner")


# Избавляемся от пропусков
df = df.dropna()

df_count = df.count()
print('Количество строк в результирующем датафрейме:')
print(df_count)

audition_count = audition_df.count()
print('Количество строк в датафрейме audition:')
print(audition_count)

content_count = content_df.count()
print('Количество строк в датафрейме content:')
print(content_count)

df.printSchema()
df.show(10)

In [ ]:
| Количество строк в результирующем датафрейме:                                                                                                                                                                                                                                                                                  |
| 297364                                                                                                                                                                                                                                                                                                                         |
| Количество строк в датафрейме audition:                                                                                                                                                                                                                                                                                        |
| 1002896                                                                                                                                                                                                                                                                                                                        |
| Количество строк в датафрейме content:                                                                                                                                                                                                                                                                                         |
| 31668 

Схема новой таблицы и первые 10 строк:

In [ ]:
| |-- main_content_id: string (nullable = true)                                                                                                                                                                                                                                                                                  |
| |-- audition_id: integer (nullable = true)                                                                                                                                                                                                                                                                                     |
| |-- puid: string (nullable = true)                                                                                                                                                                                                                                                                                             |
| |-- usage_platform_ru: string (nullable = true)                                                                                                                                                                                                                                                                                |
| |-- msk_business_dt_str: date (nullable = true)                                                                                                                                                                                                                                                                                |
| |-- app_version: string (nullable = true)                                                                                                                                                                                                                                                                                      |
| |-- adult_content_flg: boolean (nullable = true)                                                                                                                                                                                                                                                                               |
| |-- hours: float (nullable = true)                                                                                                                                                                                                                                                                                             |
| |-- hours_sessions_long: float (nullable = true)                                                                                                                                                                                                                                                                               |
| |-- kids_content_flg: boolean (nullable = true)                                                                                                                                                                                                                                                                                |
| |-- usage_geo_id_name: string (nullable = true)                                                                                                                                                                                                                                                                                |
| |-- usage_country_name: string (nullable = true)                                                                                                                                                                                                                                                                               |
| |-- main_content_type: string (nullable = true)                                                                                                                                                                                                                                                                                |
| |-- main_content_name : string (nullable = true)                                                                                                                                                                                                                                                                               |
| |-- main_content_duration_hours: double (nullable = true)                                                                                                                                                                                                                                                                      |
| |-- published_topic_title_list: string (nullable = true)                                                                                                                                                                                                                                                                       |
| |-- main_author_name: string (nullable = true)                                                                                                                                                                                                                                                                                 |
|                                                                                                                                                                                                                                                                                                                                |
| +---------------+-----------+--------------------+-----------------+-------------------+-----------+-----------------+-----------+-------------------+----------------+--------------------+------------------+-----------------+--------------------+---------------------------+--------------------------+----------------+ |
| |main_content_id|audition_id|                puid|usage_platform_ru|msk_business_dt_str|app_version|adult_content_flg|      hours|hours_sessions_long|kids_content_flg|   usage_geo_id_name|usage_country_name|main_content_type|  main_content_name |main_content_duration_hours|published_topic_title_list|main_author_name| |
| +---------------+-----------+--------------------+-----------------+-------------------+-----------+-----------------+-----------+-------------------+----------------+--------------------+------------------+-----------------+--------------------+---------------------------+--------------------------+----------------+ |
| |       R7tyI6r6|          7|68296740-f9d6-11e...|       Музыка iOS|         2024-11-26| 697.165724|             true| 0.48907945|         0.48907945|           false|           Волгоград|            Россия|        Audiobook|          Завет воды|                  30.842222|      'Художественная л...| Абрахам Вергезе| |
| |       R7tyI6r6|          8|68296740-f9d6-11e...|       Музыка iOS|         2024-11-26| 698.166333|             true| 0.18475528|          0.1847275|           false|Свердловская область|            Россия|        Audiobook|          Завет воды|                  30.842222|      'Художественная л...| Абрахам Вергезе| |
| |       QBFiEtWi|          9|6829675e-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.7|             true|  1.4530555|          1.4530555|           false|Республика Татарстан|            Россия|        Audiobook|Файролл. Книга 10...|                  20.400833|      'Художественная л...| Андрей Васильев| |
| |       xnwMA9a7|         10|6829677c-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.5|             true| 0.53998184|         0.53998184|           false|     Нижний Новгород|            Россия|             Book|Откуда берутся де...|                 10.5041275|      'Здоровье', 'Бере...|   Ася Казанцева| |
| |       jxNgtTqG|         15|682967b8-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.7|             true|  0.2905091|          0.2905091|           false|              Москва|            Россия|             Book|    Машенька. Подвиг|                     8.5502|      'Художественная л...|Владимир Набоков| |
| |       q6miekvC|         16|682967d6-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.5|             true|  2.1271636|          2.1140728|           false|              Москва|            Россия|             Book| Смертельная белизна|                  20.667345|      'Частные сыщики',...| Роберт Гэлбрейт| |
| |       xEssEuff|         17|682967d6-f9d6-11e...|  Букмейт Android|         2024-11-26|        6.5|             true| 0.13290909|         0.13290909|           false|Москва и Московск...|            Россия|             Book|Любовь и кофе. Те...|                  2.3030365|      'Поэзия', 'Мотива...|Татьяна Мужицкая| |
| |       lEpTNmxj|         24|68296812-f9d6-11e...|      Букмейт iOS|         2024-11-26|      2.5.0|             true|0.012618182|                0.0|           false|     Санкт-Петербург|            Россия|             Book|Атлант расправил ...|                  51.890854|      'Художественная л...|        Айн Рэнд| |
| |       W2qB9aGF|         25|68296812-f9d6-11e...|       Музыка iOS|         2024-11-26| 697.165724|             true|   0.683365|          0.6833372|           false|              Самара|            Россия|        Audiobook|  Одиночество в Сети|                  15.543333|      'Художественная л...| Януш Вишневский| |
| |       afv2kdL1|         30|6829684e-f9d6-11e...|  Букмейт Android|         2024-11-26|      6.4.1|             true|0.028055556|                0.0|           false|Центральный федер...|            Россия|        Audiobook|Как справиться с ...|                   7.741389|      'Ментальное здоро...| Павел Федоренко| |
| +---------------+-----------+--------------------+-----------------+-------------------+-----------+-----------------+-----------+-------------------+----------------+--------------------+------------------+-----------------+--------------------+---------------------------+--------------------------+----------------+ |
| only showing top 10 rows                                                                                                                                                                                                                                                                                                       |                                                                                                                                                                                                                                                                                                                           

Таким образом, итоговое количество строк немного уменьшилось по сравнению с исходной таблицей audition, потому что INNER JOIN исключил записи с отсутствующими соответствиями в таблице content. Также, на количество строк повлияло и то, что не все main_content_id из audition есть в content - некоторые идентификаторы контента из таблицы просмотров отсутствуют в справочнике контента.
- Количество строк в результирующем датафрейме:  297364
- Количество строк в датафрейме audition:  1002896
- Количество строк в датафрейме content:   31668 

### Удаление лишних столбцов

Удалим все лишние столбцы из объединённой таблицы, которые не нужны для дальнейшего анализа: main_author_id, app_version, usage_geo_id. Выведим первые десять строк.

In [ ]:
#Удаляем столбцы:
df = df.drop("main_author_id", "app_version", "usage_geo_id")

df.show(10)

In [ ]:
| +---------------+-----------+--------------------+-----------------+-------------------+-----------------+-----------+-------------------+----------------+--------------------+------------------+-----------------+--------------------+---------------------------+--------------------------+----------------+             |
| |main_content_id|audition_id|                puid|usage_platform_ru|msk_business_dt_str|adult_content_flg|      hours|hours_sessions_long|kids_content_flg|   usage_geo_id_name|usage_country_name|main_content_type|  main_content_name |main_content_duration_hours|published_topic_title_list|main_author_name|             |
| +---------------+-----------+--------------------+-----------------+-------------------+-----------------+-----------+-------------------+----------------+--------------------+------------------+-----------------+--------------------+---------------------------+--------------------------+----------------+             |
| |       R7tyI6r6|          7|68296740-f9d6-11e...|       Музыка iOS|         2024-11-26|             true| 0.48907945|         0.48907945|           false|           Волгоград|            Россия|        Audiobook|          Завет воды|                  30.842222|      'Художественная л...| Абрахам Вергезе|             |
| |       R7tyI6r6|          8|68296740-f9d6-11e...|       Музыка iOS|         2024-11-26|             true| 0.18475528|          0.1847275|           false|Свердловская область|            Россия|        Audiobook|          Завет воды|                  30.842222|      'Художественная л...| Абрахам Вергезе|             |
| |       QBFiEtWi|          9|6829675e-f9d6-11e...|  Букмейт Android|         2024-11-26|             true|  1.4530555|          1.4530555|           false|Республика Татарстан|            Россия|        Audiobook|Файролл. Книга 10...|                  20.400833|      'Художественная л...| Андрей Васильев|             |
| |       xnwMA9a7|         10|6829677c-f9d6-11e...|  Букмейт Android|         2024-11-26|             true| 0.53998184|         0.53998184|           false|     Нижний Новгород|            Россия|             Book|Откуда берутся де...|                 10.5041275|      'Здоровье', 'Бере...|   Ася Казанцева|             |
| |       jxNgtTqG|         15|682967b8-f9d6-11e...|  Букмейт Android|         2024-11-26|             true|  0.2905091|          0.2905091|           false|              Москва|            Россия|             Book|    Машенька. Подвиг|                     8.5502|      'Художественная л...|Владимир Набоков|             |
| |       q6miekvC|         16|682967d6-f9d6-11e...|  Букмейт Android|         2024-11-26|             true|  2.1271636|          2.1140728|           false|              Москва|            Россия|             Book| Смертельная белизна|                  20.667345|      'Частные сыщики',...| Роберт Гэлбрейт|             |
| |       xEssEuff|         17|682967d6-f9d6-11e...|  Букмейт Android|         2024-11-26|             true| 0.13290909|         0.13290909|           false|Москва и Московск...|            Россия|             Book|Любовь и кофе. Те...|                  2.3030365|      'Поэзия', 'Мотива...|Татьяна Мужицкая|             |
| |       lEpTNmxj|         24|68296812-f9d6-11e...|      Букмейт iOS|         2024-11-26|             true|0.012618182|                0.0|           false|     Санкт-Петербург|            Россия|             Book|Атлант расправил ...|                  51.890854|      'Художественная л...|        Айн Рэнд|             |
| |       W2qB9aGF|         25|68296812-f9d6-11e...|       Музыка iOS|         2024-11-26|             true|   0.683365|          0.6833372|           false|              Самара|            Россия|        Audiobook|  Одиночество в Сети|                  15.543333|      'Художественная л...| Януш Вишневский|             |
| |       afv2kdL1|         30|6829684e-f9d6-11e...|  Букмейт Android|         2024-11-26|             true|0.028055556|                0.0|           false|Центральный федер...|            Россия|        Audiobook|Как справиться с ...|                   7.741389|      'Ментальное здоро...| Павел Федоренко|             |
| +---------------+-----------+--------------------+-----------------+-------------------+-----------------+-----------+-------------------+----------------+--------------------+------------------+-----------------+--------------------+---------------------------+--------------------------+----------------+             |
| only showing top 10 rows 

### Количество пользователей (puid)

Найдём количество уникальных пользователей puid в объединённой таблице и сравним их с количеством пользователей в изначальной таблице audition_df.

In [ ]:
# Считаем количество уникальных пользователей
puid_join = df.select("puid").distinct().count()
puid_audition = audition_df.select("puid").distinct().count()

print(f"Количество уникальных пользователей в объединенной таблице: {puid_join}")
print(f"Количество уникальных пользователей в исходной таблице audition_df: {puid_audition}")

Как видно из данных, количество пользователей в объединённой таблице меньше, чем в начальной. Возможно, это связано с тем, что пользователи из начальной таблице не имеют данных в таблице с контентом. 

    Количество уникальных пользователей в объединенной таблице: 17806
    Количество уникальных пользователей в исходной таблице audition_df: 31063 

### Уникальные значения поля main_content_type.

Используя collect(), выведем на экран все уникальные значения поля main_content_type.

In [ ]:
# Получаем все уникальные значения main_content_type
unique_types = df.select("main_content_type").distinct().collect()

# Выводим их на экран
print(unique_types)

In [ ]:
Все уникальные значения main_content_type:                                                                                                                                                                                                                                                                                     |
| [Row(main_content_type='Audiobook'), Row(main_content_type='Book'), Row(main_content_type='Comicbook')]

### Полный код Шага 3

In [ ]:
# Загружаем библиотеку, для создания сессии PySpark:
from pyspark.sql import SparkSession
# Загружаем библиотеку для замены типа данных в столбцах:
from pyspark.sql.types import (StructType, StructField, LongType, StringType, FloatType, ArrayType)
# Загружаем библиотеку для создания функций в PySpark:
from pyspark.sql import functions as F

# Создаём Spark-сессию
spark = (SparkSession.builder
         .appName("DataFrame Basics")
         .config("fs.s3a.endpoint", "storage.yandexcloud.net")
         .getOrCreate())

# Указываем пути к файлам
audition = "s3a://da-plus-dags/data/audition.csv"
content = "s3a://da-plus-dags/data/content.csv"

# Считываем CSV-файл
audition_df = (spark.read
    .option("header", "false")
    .option("inferSchema", "true")
    .csv(audition))

# Считываем CSV-файл content
content_df = (spark.read
    .option("header", "false")
    .option("inferSchema", "true")
    .csv(content))

# Новые названия для столбцов
new_column_names = [
    "audition_id",
    "puid",
    "usage_platform_ru", 
    "msk_business_dt_str",
    "app_version",
    "adult_content_flg",
    "hours",
    "hours_sessions_long",
    "kids_content_flg",
    "main_content_id", 
    "usage_geo_id_name",
    "usage_country_name"
]

# Удаляем первый столбец (_c0) и переименовываем остальные
audition_df_new = audition_df.select(
    audition_df.columns[1:]  # все столбцы кроме первого (_c0)
).toDF(*new_column_names)

audition_df = audition_df_new \
    .withColumn("hours", F.col("hours").cast(FloatType())) \
    .withColumn("hours_sessions_long", F.col("hours_sessions_long").cast(FloatType()))

# Показываем результат
print("Обработанная таблица audition_df:")
audition_df.printSchema()
audition_df.show(10, truncate=False)

# Базовая статистика
audition_df.describe().show(10)

# Новые названия для столбцов
new_column_name = [
    "main_content_id",
    "main_content_type",
    "main_content_name ", 
    "main_content_duration_hours",
    "published_topic_title_list",
    "main_author_name"
]

# Переименовываем остальные столбцы
content_df = content_df.select(
    content_df.columns[0:]  # все столбцы
).toDF(*new_column_name)

# Показываем результат
print("Обработанная таблица content_df:")
content_df.printSchema()
content_df.show(10, truncate=False)

# Базовая статистика
content_df.describe().show(10)

# Присоединяем таблицы типом Semi Join, который возвращает строки из левого датафрейма, которые имеют совпадения в правом, при этом не включая данные из правого. 
df = audition_df.join(content_df, 
                                        on="main_content_id", 
                                        how="inner")


# Избавляемся от пропусков
df = df.dropna()

df_count = df.count()
print('Количество строк в результирующем датафрейме:')
print(df_count)

audition_count = audition_df.count()
print('Количество строк в датафрейме audition:')
print(audition_count)

content_count = content_df.count()
print('Количество строк в датафрейме content:')
print(content_count)

df.printSchema()
df.show(10)


#Удаляем столбцы:
df = df.drop("main_author_id", "app_version", "usage_geo_id")

df.show(10)

# Считаем количество уникальных пользователей
puid_join = df.select("puid").distinct().count()
puid_audition = audition_df.select("puid").distinct().count()

print(f"Количество уникальных пользователей в объединенной таблице: {puid_join}")
print(f"Количество уникальных пользователей в исходной таблице audition_df: {puid_audition}")

# Получаем все уникальные значения main_content_type
unique_types = df.select("main_content_type").distinct().collect()

# Выводим их на экран
print("Все уникальные значения main_content_type:")
print(unique_types)

## Выводы

В данной работе необходимо обработать и проанализировать реальные пользовательские данные таблиц  (source_db.audition) и (source_db.content) из сервиса Яндекс Книги с помощью PySpark.
    Из расчётов были получены следующие данные:
- В таблице audition содержимое столбцов и их тип не совподает с предпологаемым. В таблице 12 столбцов и 1002895 строк. Но названия в столбцах отсутствует и лишний один столбец (первый).  В таблице audition есть лишний столбец с типом integer (nullable = true). И нет названий у сталбцов. Также не совпадают типы данных в столбцах  **hours** (ожидался: Float32, а фактически: double (Float64)), **hours_sessions_long**(ожидался: Float32, а фактически: double (Float64)), **app_version**(ожидался: Nullable(String), фактически: string).
- В таблице content содержимое столбцов и их тип не совподает с предпологаемым. В таблице 6 столбцов и 31667  строк. Но названия у столбцов отсутствуют. В таблице content типы данных у столбцов  **main_content_duration_hours** (ожидался Float32, а фактически double), для **published_topic_title_list**(ожидался Array(String), а фактически string)) не совпадают. И также нет названий у сталбцов. И столбцы **main_author_name** и **published_topic_title_list** поменяны местами.
- Все уникальные значения main_content_type:Row(main_content_type='Audiobook'), Row(main_content_type='Book'), Row(main_content_type='Comicbook')
- Количество уникальных пользователей в объединенной таблице: 17806
- Количество уникальных пользователей в исходной таблице audition_df: 31063

Обнаруженная разница в количестве строк, скорее всего, обусловлено тем, что в таблице audition хранятся данные о пользователях, а в таблице content о контенте. Так как у одной книги может быть сотни и даже сотни тысяч читателей, то и связана одна книга из одной таблицы может с тысячами пользователями из другой. 
    Из расчётов в пункте 5.3-5.4, видно, что больше всего сумма часов сессий в рабочие дни(17997378), чем в выходные (6599755). Это говорит о том, что пользователи предпочитаеют пользоваться приложением на рабочей неделе, чем на выходных.
    Также, в результате анализа было выявлено, что суммарное количество часов сессий больше у контента для "взрослых"(19 823 755 часов), чем у контента с рейтингом "детский"(4 773 378 часов). Так результаты расчётов показали, что чаще всего пользователи посещают приложение в будние дни, что с контентом для взрослых(на буднях 14625640 часов, а на выходных 5198115 часов), что для детей (на буднях 3371738 часов, а на выходных 1401640 часов).
    из пункта 6.3 видно, что количество пользователей в объединённой таблице меньше, чем в начальной. Возможно, это связано с тем, что пользователи из начальной таблице не имеют данных в таблице с контентом.     

In [ ]:
spark.stop()